In [1]:
from pymongo import MongoClient

In [2]:
client = MongoClient('localhost') #계정 정보를 가지고 접속하려면. 여기다 id, pw ,db 에 대해서 명시가능. shift + tab 에는 명시가 안되어있지만 kwargs 에 id pw 를 넣을수있다는것임.

In [3]:
client.list_database_names()

['admin', 'config', 'local', 'newDB', 'sns', 'test']

In [4]:
client.sns.get_collection('user')

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'sns'), 'user')

In [5]:
client.sns.get_collection('feed')

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'sns'), 'feed')

In [6]:
def create_user(user_id,user_pw,name,email):
    new_user  = {'_id':user_id,
                 'pw':user_pw,
                 'name':name,
                 'email':email,
                 'address':[],
                 'profile':"",
                 'friends':[]
                }
    client.sns.user.insert_one(new_user)


In [7]:
create_user(5,"adv123","김정민","jeungmin717@gmail.com")

In [15]:
def add_address(user_id,address,description,title):
    new_address={
        'address':address,
        'description':description,
        'title':title
    }#임베디드 다큐먼트 형으로 insert.
    client.sns.user.update_one({'_id':user_id},{'$addToSet':{'address':new_address}})  #앞의 괄호 {} : 이건 아무조건없는거 , 뒤의 괄호 수정할것의 조건
    # list 안에다가 새로운 list 를 계속해서 넣어주는것.  그것은 $set 이 아니라 $addToSet 이다. #set 하면  기존값을 바꿔버림.

In [19]:
add_address(1,"서울시 노원구 월계동 508-32","sk view 아파트 맞은편","집")

주소를 지우는 서비스를 만들자

In [31]:
def delete_address(user_id,address):
    return client.sns.user.update_one(
    {'_id':user_id},
    {'$pull':{'address':{'address':address}}}) # user 에서 , address.address 는 address 가 가지고있는address 다 . 를 delete

In [32]:
result = delete_address(1,"서울시 성북구")

In [33]:
result.raw_result

{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}

In [27]:
# 조건에 만족하는것을 n:1 1개 찾음. 하지만 nModified : 0 수정된것은 0 없다

In [36]:
validator = {
    '$JsonSchema':{
        'bsonType':'object',
        'required':['name','member'],
        'properties':{
            'name':{
                'bsonType':'string',
                'description':'string'
            },
            'memeber':{
                'bsonType':'array',
                'description':'Memeber must be array'
            }
        }
    }
}

#jsonSchema 형태로 validator 를 만들고 컬렉션을 생성할때 ,schema 를 멕인다


In [37]:
client.sns.get_collection('groupp',{'validator':validator})

TypeError: codec_options must be an instance of bson.codec_options.CodecOptions